In [ ]:
import os
import requests
from dotenv import load_dotenv
import gradio as gr
import re

load_dotenv()

# Load environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
AI_SEARCH_SEMANTIC = os.getenv("AI_SEARCH_SEMANTIC")
AI_SEARCH_KEY = os.getenv("AI_SEARCH_KEY")
AI_SEARCH_INDEX = os.getenv("AI_SEARCH_INDEX")

# Function to extract a limited number of message history for GPT request
def get_history_messages(histories):
    history_list = list()

    history_length = 5  # Define maximum number of message pairs to keep
    history_index = 0

    # Iterate through the history of messages
    for history in histories:
        
        if history_index >= history_length:
            break
        # Separate user and assistant messages
        message1 = history[0]
        message2 = history[1]
        
        # Add the messages to the list in a structured format
        history_list.append({
            "role": "assistant",
            "content": message1
        })
        history_list.append({
            "role": "assistant",
            "content": message2               
        })
        
        history_index += 1
        
    return history_list

# Function to request GPT for a response
def request_gpt(prompt, history_list):
    headers = {"Content-Type": "application/json", "api-key": AZURE_OPENAI_API_KEY}
    message_list = list()
    
    # System role message to instruct GPT
    message_list.append({
        "role": "system",
        "content": f"""
                    You are an assistant for medical professionals. 
                    When asked about a disease, search the "disease" section of the provided data. If found, answer based on the document. 
                    If not, inform the user that no document is available for that disease.
                """
    })
    # Add message history to the payload
    message_list.extend(history_list)
    # Add the user prompt to the payload
    message_list.append({
        "role": "user",
        "content":prompt
    })
    # Define the payload with GPT settings and Azure Search configuration
    payload = {
        "messages": message_list,
        "temperature": 0.1,
        "top_p": 0.6,
        "max_tokens": 800,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": AI_SEARCH_ENDPOINT,
                "semantic_configuration": AI_SEARCH_SEMANTIC,
                "query_type": "semantic",
                "strictness": 5,
                "top_n_documents": 5,
                "key": AI_SEARCH_KEY,
                "indexName": AI_SEARCH_INDEX
            }
        }
        ]
    }
    
    response = requests.post(AZURE_OPENAI_ENDPOINT, headers=headers, json=payload)
    
    if response.status_code == 200:
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        if content == "The requested information is not available in the retrieved data. Please try another query or topic.":
            return content, None
        # Check if there are any citations in the response
        if response_json["choices"][0]["message"]["context"]:
            citations = response_json["choices"][0]["message"]["context"]["citations"]
            formatted_citation_list = list()
            i = 0
            for c in citations:
                i += 1
                temp = f"<details><summary>Doc{i}</summary><ul>{c['content']}</ul></details>"
                formatted_citation_list.append(temp)
                
        else:
            formatted_citation_list = list() # No citations
            
        return content, "".join(formatted_citation_list) # Return response content and formatted citations
    else:
        return f"{response.status_code}, {response.text}", ""

# Function to handle user prompt submission
def click_send(prompt, histories):
    # Retrieve message history for the GPT request
    history_list = get_history_messages(histories=histories)
    # Send the prompt and history to GPT and get the response
    response_text, citation_html = request_gpt(prompt, history_list)
    histories.append((prompt, response_text))
    return histories, "", citation_html



### Web UI ###
with gr.Blocks(css="styles.css") as demo:
    
    # Main Interaction Area
    with gr.Row():
        with gr.Column(scale=3):   
            # Chatbot, Citation area
            with gr.Row(elem_id="chatbot-container"):
                chatbot = gr.Chatbot(label="Chat history", elem_classes="chatbot")
                citation = gr.HTML(label="reference area", elem_classes="citation-box")
            # Input Box and Submit Button
            with gr.Row(elem_id="input-container"):
                input_openai_textbox = gr.Textbox(label="", elem_id="textbox", scale=7, placeholder="질문을 입력하세요...")
                send_button = gr.Button("Submit", elem_id="button", scale=1)
            
            # Connect the chatbot and button functionality
            input_openai_textbox.submit(fn=click_send, inputs=[input_openai_textbox, chatbot], outputs=[chatbot, input_openai_textbox, citation])
            send_button.click(fn=click_send, inputs=[input_openai_textbox, chatbot], outputs=[chatbot, input_openai_textbox, citation])

        
demo.launch()


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\components\chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
